# Data preprocessing and Feature Engineering

In order to obtain a learnable dataset, features must be preprocessed and engineered in order to contain valuable learning data. This notebook will generate a PySpark RDD that contains the preprocessed dataset, ready for feature selection algorithms.

More than 20 features must be engineered, some ideas are:

Session-time related features:
* Weekday of the session
* Month of the session
* Season of the session
* Duration of the session
* Day period of the session

Session-item related features:
* Item with the most time spent on.
* Mean time per item.
* Variance time per item.
* Item revisit?

### Starting the Spark engine and loading the dataset.

In [1]:
import os 
import warnings
warnings.filterwarnings('ignore')
import pandas as pd

# launch this cell if you have issues on windows with py4j (think about updating your PATH)
import sys
os.environ['PYSPARK_DRIVER_PYTHON_OPTS']= "notebook"
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable
os.environ['PYSPARK_PYTHON'] = sys.executable

# starts a spark session from notebook

os.environ['PYSPARK_SUBMIT_ARGS'] ="--conf spark.driver.memory=2g  pyspark-shell"
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("load_explore") \
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/10 11:50:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
# loads relevant datas in DataFrames
train_sessions = spark.read.load('../Data/train_sessions.csv', 
                          format='com.databricks.spark.csv', 
                          header='true', 
                          inferSchema='true')

train_purchases = spark.read.load('../Data/train_purchases.csv', 
                          format='com.databricks.spark.csv', 
                          header='true', 
                          inferSchema='true')

candidate_items = spark.read.load('../Data/candidate_items.csv', 
                          format='com.databricks.spark.csv', 
                          header='true', 
                          inferSchema='true')

item_features = spark.read.load('../Data/item_features.csv', 
                          format='com.databricks.spark.csv', 
                          header='true', 
                          inferSchema='true')

datas = [train_sessions, train_purchases, candidate_items, item_features]

## Group sessions and aggregate their duration.

Using a map reduce operation, we will start by computing the session duration in seconds. For that, we use a function that will convert the timestamp into seconds since the 1st January 1970 (UNIX time).

In [3]:
import datetime

def timestamp_to_unix(timestamp: str) -> int:
    '''Converts the timestamp, on a string format to the UNIX time
    '''
    try:
        date = datetime.datetime.strptime(timestamp, '%Y-%m-%d %H:%M:%S.%f')
    except ValueError:
        date = datetime.datetime.strptime(timestamp, '%Y-%m-%d %H:%M:%S')
    return int(date.timestamp())

mapped_sessions = train_sessions.rdd.map(lambda x: (x.session_id, timestamp_to_unix(x.date))) 
sampled_sessions = mapped_sessions.sample(False, 0.0001, seed=42)

In [4]:
sampled_sessions.take(1)

[(4163, 1606579200)]

In [6]:
def reduce_min_max(x, y):
    if type(x) == tuple:
        return(max(x[0], y), min(x[1], y))
    return (max(x, y), min(x, y))

reduced_sessions = mapped_sessions.reduceByKey(lambda x, y: reduce_min_max(x, y))
reduced_sessions.take(1)

[(24, (1582741472, 1582737768))]

In [20]:
def get_duration_seconds(tupl):
    if type(tupl[1]) is int:
        return (tupl[0], 0)
    else:
        return (tupl[0], tupl[1][0] - tupl[1][1])

time_sessions = reduced_sessions.map(lambda x: get_duration_seconds(x))

In [21]:
time_sessions.take(50)

[(24, 3704),
 (28, 87),
 (36, 43),
 (44, 33),
 (48, 657),
 (52, 38),
 (108, 1473),
 (124, 0),
 (140, 173),
 (156, 0),
 (184, 0),
 (188, 0),
 (208, 0),
 (232, 0),
 (248, 1716),
 (324, 28),
 (332, 21),
 (340, 268),
 (352, 190),
 (376, 171),
 (380, 30396),
 (384, 409),
 (388, 0),
 (412, 920),
 (428, 567),
 (464, 0),
 (480, 393),
 (496, 394),
 (508, 296),
 (512, 964),
 (540, 1487),
 (544, 2320),
 (556, 309),
 (564, 0),
 (592, 524),
 (632, 20),
 (656, 0),
 (688, 183),
 (696, 27),
 (712, 97),
 (720, 23944),
 (756, 0),
 (808, 1233),
 (824, 0),
 (832, 10388),
 (844, 45),
 (864, 0),
 (884, 251),
 (896, 147),
 (900, 40085)]